# Optimización de un modelo de red neuronal (fully-connected)

Este notebook recoge los resultados de la búsqueda del mejor modelo de clasificación mediante una red neuronal densa o fully-connected, ya que el uso de redes convolucionales no parece adecuado para un problema de datos tabulares.

Para buscar el mejor modelo posible, se tratará de buscar los mejores hiperparámetros para el número de capas ocultas de la red, su anchura (número de neuronas), posible introducción de términos de regularización, optimizadores, ...

### Preparación de los datos

In [1]:
import pandas as pd
import numpy as np
# Data partition
from sklearn.model_selection import train_test_split
# Parameter tunning libraries
import optuna
from sklearn.model_selection import GridSearchCV
# Accuracy function
from sklearn.metrics import accuracy_score

In [2]:
# Datos de entrenamiento
trainFNC = pd.read_csv("../data/train_FNC.csv")
trainSBM = pd.read_csv("../data/train_SBM.csv")
train_labels = pd.read_csv("../data/train_labels.csv")

# DataFrame con ambas fuentes de datos
train = pd.merge(left=trainFNC, right=trainSBM, left_on='Id', right_on='Id')
data = pd.merge(left=train_labels, right=train, left_on='Id', right_on='Id')
data.drop("Id", inplace=True, axis=1)

# Shuffle de los datos de train
data = data.sample(frac=1, random_state=0)
data.head(5)

,Class,FNC1,FNC2,FNC3,FNC4,FNC5,FNC6,FNC7,FNC8,FNC9,...,SBM_map55,SBM_map61,SBM_map64,SBM_map67,SBM_map69,SBM_map71,SBM_map72,SBM_map73,SBM_map74,SBM_map75
2,0,0.245850,0.216620,-0.124680,-0.353800,0.161500,-0.002032,-0.133020,-0.035222,0.259040,...,-0.257114,0.597229,1.220756,-0.059213,-0.435494,-0.092971,1.090910,-0.448562,-0.508497,0.350434
13,1,0.410730,-0.031925,0.210700,0.242260,0.320100,-0.419290,-0.187140,0.168450,0.599790,...,-0.050862,0.870602,0.609465,1.181878,-2.279469,-0.013484,-0.012693,-1.244346,-1.080442,-0.788502
53,1,0.070919,0.034179,-0.011755,0.019158,0.024645,-0.032022,0.004620,0.318170,0.212550,...,-1.539922,-1.495822,1.643866,1.687780,1.521086,-1.988432,-0.267471,0.510576,1.104566,-1.067206
41,0,0.087377,-0.052462,-0.007835,-0.112830,0.389380,0.216080,0.063572,-0.251230,-0.080568,...,-0.077353,-0.459463,-0.204328,-0.619508,-1.410523,-0.304622,-1.521928,0.593691,0.073638,-0.260920
74,0,0.202750,0.191420,-0.056662,-0.157780,0.244040,0.039780,-0.001503,0.001056,-0.048222,...,0.044457,0.593326,1.063052,0.434726,1.604964,-0.359736,0.210107,0.355922,0.730287,-0.323557


Vamos a usar la siguiente partición de los datos:

* 60% train $\sim$ 50 datos
* 20% validation $\sim$ 18 datos (se define al aplicar cross-validación en el ajuste)
* 20% test $\sim$ 18 datos

In [3]:
X = data.iloc[:, 1:]
y = data.iloc[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Tamaño del dataset de train:", X_train.shape)
print("Tamaño del dataset de test:", X_test.shape)

Tamaño del dataset de train: (68, 410)
Tamaño del dataset de test: (18, 410)


In [4]:
# Datos de test
testFNC = pd.read_csv("../data/test_FNC.csv")
testSBM = pd.read_csv("../data/test_SBM.csv")

# DataFrame con ambas fuentes de datos
test = pd.merge(left=testFNC, right=testSBM, left_on='Id', right_on='Id')
test.drop("Id", inplace=True, axis=1)
test.head(5)

,FNC1,FNC2,FNC3,FNC4,FNC5,FNC6,FNC7,FNC8,FNC9,FNC10,...,SBM_map55,SBM_map61,SBM_map64,SBM_map67,SBM_map69,SBM_map71,SBM_map72,SBM_map73,SBM_map74,SBM_map75
0,0.476127,0.064466,0.053238,-0.608133,0.073988,-0.637038,0.113556,-0.192434,-0.004025,-0.060474,...,-0.451994,1.123770,2.083006,1.145440,-0.067608,1.202529,0.851587,0.451583,-0.159739,0.192076
1,0.013833,0.267183,0.232178,-0.167151,-0.261327,0.191869,0.406493,0.088761,0.177048,0.036718,...,0.696987,1.397832,1.046136,-0.191733,-2.192023,-0.369276,0.822225,-0.109342,-0.580476,0.174160
2,-0.435452,0.046780,0.243742,0.397030,-0.147821,0.173620,-0.461963,-0.610736,0.419753,0.400985,...,0.160145,1.906989,-2.661633,-0.193911,0.440873,0.641739,0.918397,-0.758046,0.154701,-0.476647
3,-0.204510,-0.036735,-0.760705,-0.740495,0.064668,0.349926,-0.273826,-0.174384,-0.120248,0.175618,...,0.974828,-1.997087,-2.083782,1.154107,-0.643947,2.332424,0.659124,-0.809445,0.558960,2.790871
4,0.599435,-0.166441,0.122431,0.011539,0.346906,-0.017430,-0.274734,0.211510,0.151012,-0.033434,...,-0.789153,1.578984,1.402592,-1.230440,0.296686,2.806314,0.427184,-0.240682,-0.196948,-1.544345


### Modelo

Para redes neuronales, compararemos los resultados obtenidos construyendo redes a partir de librerías distintas.

**Comenzamos con ``MLPClassifier`` de ``sklearn`` y búsqueda de hiperparámetros con ``GridSearchCV``:**

Documentación: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

In [10]:
from sklearn.neural_network import MLPClassifier

In [12]:
def train_model(model, param_grid):
    '''Función para realizar el entrenamiento y la búsqueda de hiperparámetros'''
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=4)
    # cv = 4 porque así: el conjunto de validation tiene un 0.25 del tamaño de train y: 0.25 * 0.8 = 0.2 ~ 20% datos
    #                    el conjunto de train tiene un 0.75 del tamaño de train y: 0.75 * 0.8 = 0.6 ~60% datos
    grid_search.fit(X_train, y_train)
    
    print("Parámetros óptimos:", grid_search.best_params_)
    print("Modelo óptimo:", grid_search.best_estimator_)
    
    return grid_search.best_estimator_

El método debe recibir arquitecturas de red pre-definidas, por lo que probaremos topologías variadas en cuanto a profundidad, ancho y número de capas.

In [15]:
# import warnings
# warnings.filterwarnings("ignore")

# Definir y entrenar el modelo
model_MLPC = MLPClassifier(max_iter=1000, random_state=0)
param_grid_MLPC = {
    "hidden_layer_sizes": [(100, 200, 100, 1), (100, 100, 100, 100, 1), (200, 200, 100, 50, 1), (100, 250, 250, 100, 1)],
    "activation": ["identity", "logistic", "tanh", "relu"], # Tiene sentido probar identity y relu?
    "solver": ["lbfgs", "sgd", "adam"],
#     "alpha": [0, 0.0001, 0.001, 0.01, 0.1], # L2 regularization
#     "learning_rate": ["constant", "invscaling", "adaptive"],
    "validation_fraction": [0.25]
}
model_MLPC_opt = train_model(model_MLPC, param_grid_MLPC)

# Predicción en partición de test
y_pred_MLPC = model_MLPC_opt.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_MLPC)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

# Predicción en test para kaggle
y_pred_kaggle_MLPC = model_MLPC_opt.predict(test)

C:\Users\saral\miniconda3\envs\TFM\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\saral\miniconda3\envs\TFM\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\saral\miniconda3\envs\TFM\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\saral\miniconda3\envs\TFM\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\saral\miniconda3\en

C:\Users\saral\miniconda3\envs\TFM\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\saral\miniconda3\envs\TFM\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\saral\miniconda3\envs\TFM\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\saral\miniconda3\envs\TFM\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\saral\miniconda3\en

Parámetros óptimos: {'activation': 'relu', 'hidden_layer_sizes': (100, 250, 250, 100, 1), 'solver': 'adam', 'validation_fraction': 0.25}
Modelo óptimo: MLPClassifier(hidden_layer_sizes=(100, 250, 250, 100, 1), max_iter=1000,
              random_state=0, validation_fraction=0.25)
Accuracy: 83.33%


**Usando la librería "keras"**

Ahora utilizaremos la librería de ``keras``, por su mayor flexibilidad para intentar mejorar los resultados de la red neuronal.

Comenzaremos repitiendo la búsqueda de hiperparámetros, ya que la propia librería de ``keras`` dispone de integración con otras que nos permitirán hacer una búsqueda algo más exhaustiva por ejemplo en cuanto al número de capas y neuronas en estas. Concretamente, vamos a utilizar ``optuna``.

In [16]:
import tensorflow as tf
import keras
from keras import layers, models, optimizers, callbacks, backend, preprocessing

Documentación:
* https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html
* https://optuna.org/

Para reducir el coste computacional tomaremos de base resultados como la función de activación óptima: "relu", que hemos podido obtener con ``GridSearchCV``. Como por el contrario usando ``sklearn`` no hemos podido utilizar el optimizador RMSProp, vamos a probarlo también con ``optuna`` + ``keras`` para ver si mejora nuestros resultados.

In [17]:
# El objetivo es definir una función que será optimizada. En este caso, nos interesa maximizar el accuracy.
def objective(trial):
    modelFC_optuna = models.Sequential()

    # Se utiliza el objeto "trial" para asignar las posibilidades a los hiperparámetros.
    n_layers = trial.suggest_int("n_layers", 2, 5, 1)
    num_hidden = trial.suggest_int("n_units", 50, 250, 50)
    dropout = trial.suggest_float("dropout", 0, 0.5, step=0.1)
    for i in range(n_layers):
        modelFC_optuna.add(layers.Dense(num_hidden, activation="relu"))
        modelFC_optuna.add(layers.Dropout(rate=dropout))
    modelFC_optuna.add(layers.Dense(1, activation="sigmoid"))

    optimizers = trial.suggest_categorical("optimizer", ["RMSprop", "SGD", "Adam"])
    modelFC_optuna.compile(loss="categorical_crossentropy", optimizer=optimizers, metrics=["accuracy"])
    
    es = callbacks.EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience=5)
    modelFC_optuna.fit(X_train, y_train, callbacks=[es], epochs=100, validation_split=0.25, verbose=0)

    loss, accuracy = modelFC_optuna.evaluate(X_test, y_test)
    return accuracy

In [19]:
# Creamos un objeto "study" y buscamos la optimización de la función objetivo.
sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=360)
# n_trials = 4 x 5 x 6 x 3 = 360

[I 2022-05-24 00:26:13,347] A new study created in memory with name: no-name-f3a42e01-b56d-463e-8226-61b1ff20155a


1/1 [==============================] - 0s 31ms/step - loss: 0.0000e+00 - accuracy: 0.4444


[I 2022-05-24 00:26:21,044] Trial 0 finished with value: 0.4444444477558136 and parameters: {'n_layers': 4, 'n_units': 200, 'dropout': 0.30000000000000004, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.4444444477558136.


1/1 [==============================] - 0s 37ms/step - loss: 0.0000e+00 - accuracy: 0.4444


[I 2022-05-24 00:26:23,436] Trial 1 finished with value: 0.4444444477558136 and parameters: {'n_layers': 3, 'n_units': 250, 'dropout': 0.5, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.4444444477558136.


...

[I 2022-05-24 00:35:17,623] Trial 357 finished with value: 0.4444444477558136 and parameters: {'n_layers': 3, 'n_units': 250, 'dropout': 0.0, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.4444444477558136.


1/1 [==============================] - 0s 31ms/step - loss: 0.0000e+00 - accuracy: 0.4444


[I 2022-05-24 00:35:18,965] Trial 358 finished with value: 0.4444444477558136 and parameters: {'n_layers': 3, 'n_units': 250, 'dropout': 0.30000000000000004, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.4444444477558136.


1/1 [==============================] - 0s 16ms/step - loss: 0.0000e+00 - accuracy: 0.4444


[I 2022-05-24 00:35:20,190] Trial 359 finished with value: 0.4444444477558136 and parameters: {'n_layers': 3, 'n_units': 250, 'dropout': 0.30000000000000004, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.4444444477558136.


In [20]:
study.best_trial

FrozenTrial(number=0, values=[0.4444444477558136], datetime_start=datetime.datetime(2022, 5, 24, 0, 26, 13, 348986), datetime_complete=datetime.datetime(2022, 5, 24, 0, 26, 21, 28622), params={'n_layers': 4, 'n_units': 200, 'dropout': 0.30000000000000004, 'optimizer': 'Adam'}, distributions={'n_layers': IntUniformDistribution(high=5, low=2, step=1), 'n_units': IntUniformDistribution(high=250, low=50, step=50), 'dropout': DiscreteUniformDistribution(high=0.5, low=0.0, q=0.1), 'optimizer': CategoricalDistribution(choices=('RMSprop', 'SGD', 'Adam'))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE, value=None)

La red con los parámetros optimizados es la siguiente:

In [37]:
tf.keras.utils.set_random_seed(0) # Reproducibilidad de resultados

# Definir y entrenar el modelo
modelFC_optuna = models.Sequential()
modelFC_optuna.add(layers.Dense(200, activation="relu", input_shape=(410,)))
modelFC_optuna.add(layers.Dropout(0.3))
modelFC_optuna.add(layers.Dense(200, activation="relu"))
modelFC_optuna.add(layers.Dropout(0.3))
modelFC_optuna.add(layers.Dense(200, activation="relu"))
modelFC_optuna.add(layers.Dropout(0.3))
modelFC_optuna.add(layers.Dense(200, activation="relu"))
modelFC_optuna.add(layers.Dropout(0.3))
modelFC_optuna.add(layers.Dense(1, activation="sigmoid"))

modelFC_optuna.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])
es = callbacks.EarlyStopping(monitor="val_acc", min_delta=0.01, patience=5)
modelFC_optuna.fit(X_train, y_train, epochs=100, validation_split=0.25, callbacks=[es])
# modelFC_optuna_Adam.fit(X_train, y_train, epochs=100, validation_split=0.25)

# Precisión en partición de test
loss, accuracy = modelFC_optuna.evaluate(X_test, y_test)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Epoch 1/100
2/2 [==============================] - 1s 173ms/step - loss: 0.6907 - acc: 0.5686 - val_loss: 0.7020 - val_acc: 0.4706
Epoch 2/100
2/2 [==============================] - 0s 31ms/step - loss: 0.6908 - acc: 0.5490 - val_loss: 0.6967 - val_acc: 0.4118
Epoch 3/100
2/2 [==============================] - 0s 38ms/step - loss: 0.6219 - acc: 0.7255 - val_loss: 0.6908 - val_acc: 0.3529
Epoch 4/100
2/2 [==============================] - 0s 27ms/step - loss: 0.6492 - acc: 0.6471 - val_loss: 0.6842 - val_acc: 0.5294
Epoch 5/100
2/2 [==============================] - 0s 31ms/step - loss: 0.6124 - acc: 0.6275 - val_loss: 0.6722 - val_acc: 0.5882
Epoch 6/100
2/2 [==============================] - 0s 32ms/step - loss: 0.5017 - acc: 0.8235 - val_loss: 0.6597 - val_acc: 0.6471
Epoch 7/100
2/2 [==============================] - 0s 27ms/step - loss: 0.5774 - acc: 0.6667 - val_loss: 0.6501 - val_acc: 0.6471
Epoch 8/100
2/2 [==============================] - 0s 31ms/step - loss: 0.4641 - acc: 0.8

$\color{red}{\text{SI NO FIJAMOS SEMILLA ALGUNAS EJECUCIONES SI MEJORAN EL RESULTADO DE sklearn}}$

Veamos si podemos obtener mejores resultados cambiando la última capa con activación sigmoide por una activación softmax:

In [38]:
from keras.utils import np_utils

# En primer lugar, hay que adaptar los datos
NUM_CLASSES = 2
y_train_softmax = np_utils.to_categorical(y_train, NUM_CLASSES)
y_test_softmax = np_utils.to_categorical(y_test, NUM_CLASSES)

In [42]:
def objectiveSoftmax(trial):
    modelFC_optuna = models.Sequential()

    n_layers = trial.suggest_int("n_layers", 2, 5, 1)
    num_hidden = trial.suggest_int("n_units", 50, 250, 50)
    dropout = trial.suggest_float("dropout", 0, 0.5, step=0.1)
    for i in range(n_layers):
        modelFC_optuna.add(layers.Dense(num_hidden, activation="relu"))
        modelFC_optuna.add(layers.Dropout(rate=dropout))
    modelFC_optuna.add(layers.Dense(2, activation="softmax"))

    optimizers = trial.suggest_categorical("optimizer", ["RMSprop", "SGD", "Adam"])
    modelFC_optuna.compile(loss="categorical_crossentropy", optimizer=optimizers, metrics=["accuracy"])
    
    es = callbacks.EarlyStopping(monitor="val_accuracy", min_delta=0.01, patience=5)
    modelFC_optuna.fit(X_train, y_train_softmax, callbacks=[es], epochs=100, validation_split=0.25, verbose=0)

    loss, accuracy = modelFC_optuna.evaluate(X_test, y_test_softmax)
    return accuracy

In [43]:
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objectiveSoftmax, n_trials=360)
# n_trials = 4 x 5 x 6 x 3 = 360

[I 2022-05-24 01:03:51,948] A new study created in memory with name: no-name-4e39b162-3654-491b-bdab-6ca7496aa86a


1/1 [==============================] - 0s 16ms/step - loss: 0.6922 - accuracy: 0.3889


[I 2022-05-24 01:03:53,243] Trial 0 finished with value: 0.3888888955116272 and parameters: {'n_layers': 5, 'n_units': 50, 'dropout': 0.5, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.3888888955116272.


1/1 [==============================] - 0s 16ms/step - loss: 0.7026 - accuracy: 0.5000


[I 2022-05-24 01:03:54,548] Trial 1 finished with value: 0.5 and parameters: {'n_layers': 5, 'n_units': 200, 'dropout': 0.4, 'optimizer': 'SGD'}. Best is trial 1 with value: 0.5.


...

[I 2022-05-24 01:11:23,660] Trial 357 finished with value: 0.5555555820465088 and parameters: {'n_layers': 5, 'n_units': 250, 'dropout': 0.1, 'optimizer': 'SGD'}. Best is trial 27 with value: 0.8888888955116272.


1/1 [==============================] - 0s 31ms/step - loss: 0.7053 - accuracy: 0.7222


[I 2022-05-24 01:11:25,149] Trial 358 finished with value: 0.7222222089767456 and parameters: {'n_layers': 5, 'n_units': 150, 'dropout': 0.0, 'optimizer': 'RMSprop'}. Best is trial 27 with value: 0.8888888955116272.


1/1 [==============================] - 0s 34ms/step - loss: 0.5447 - accuracy: 0.6111


[I 2022-05-24 01:11:26,770] Trial 359 finished with value: 0.6111111044883728 and parameters: {'n_layers': 5, 'n_units': 50, 'dropout': 0.1, 'optimizer': 'RMSprop'}. Best is trial 27 with value: 0.8888888955116272.


In [44]:
study.best_trial

FrozenTrial(number=27, values=[0.8888888955116272], datetime_start=datetime.datetime(2022, 5, 24, 1, 4, 24, 625763), datetime_complete=datetime.datetime(2022, 5, 24, 1, 4, 26, 73405), params={'n_layers': 4, 'n_units': 150, 'dropout': 0.1, 'optimizer': 'RMSprop'}, distributions={'n_layers': IntUniformDistribution(high=5, low=2, step=1), 'n_units': IntUniformDistribution(high=250, low=50, step=50), 'dropout': DiscreteUniformDistribution(high=0.5, low=0.0, q=0.1), 'optimizer': CategoricalDistribution(choices=('RMSprop', 'SGD', 'Adam'))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=27, state=TrialState.COMPLETE, value=None)

La red con los parámetros optimizados es la siguiente:

In [45]:
tf.keras.utils.set_random_seed(0)

# Definir y entrenar el modelo
modelFC_optuna = models.Sequential()
modelFC_optuna.add(layers.Dense(150, activation="relu", input_shape=(410,)))
modelFC_optuna.add(layers.Dropout(0.1))
modelFC_optuna.add(layers.Dense(150, activation="relu"))
modelFC_optuna.add(layers.Dropout(0.1))
modelFC_optuna.add(layers.Dense(150, activation="relu"))
modelFC_optuna.add(layers.Dropout(0.1))
modelFC_optuna.add(layers.Dense(150, activation="relu"))
modelFC_optuna.add(layers.Dropout(0.1))
modelFC_optuna.add(layers.Dense(2, activation="softmax"))

modelFC_optuna.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])
es = callbacks.EarlyStopping(monitor="val_acc", min_delta=0.01, patience=5)
modelFC_optuna.fit(X_train, y_train_softmax, epochs=100, validation_split=0.25, callbacks=[es])

# Precisión en partición de test
loss, accuracy = modelFC_optuna.evaluate(X_test, y_test_softmax)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Epoch 1/100
2/2 [==============================] - 1s 241ms/step - loss: 0.6755 - acc: 0.5882 - val_loss: 0.6444 - val_acc: 0.5882
Epoch 2/100
2/2 [==============================] - 0s 41ms/step - loss: 0.6054 - acc: 0.6275 - val_loss: 0.6108 - val_acc: 0.6471
Epoch 3/100
2/2 [==============================] - 0s 44ms/step - loss: 0.4580 - acc: 0.8627 - val_loss: 0.5695 - val_acc: 0.7059
Epoch 4/100
2/2 [==============================] - 0s 38ms/step - loss: 0.3061 - acc: 0.9216 - val_loss: 0.5952 - val_acc: 0.7059
Epoch 5/100
2/2 [==============================] - 0s 36ms/step - loss: 0.2151 - acc: 0.9804 - val_loss: 0.5713 - val_acc: 0.7647
Epoch 6/100
2/2 [==============================] - 0s 33ms/step - loss: 0.1228 - acc: 1.0000 - val_loss: 0.5352 - val_acc: 0.7059
Epoch 7/100
2/2 [==============================] - 0s 34ms/step - loss: 0.0535 - acc: 1.0000 - val_loss: 0.6135 - val_acc: 0.7059
Epoch 8/100
2/2 [==============================] - 0s 38ms/step - loss: 0.0545 - acc: 1.0

Obtenemos un mayor accuracy utilizando la activación softmax. Sin embargo, parece que ahora la red tiene un problema de sobreajuste, ya que por ejemplo en la última época hay una diferencia entre la precisión en train y en validación del 30% aproximadamente. Incluso, desde la 6ª etapa, la red clasifica los datos en train con una precisión del 100%. Vamos a tratar de reducir esta diferencia probando distintos tipos de regularización.

$\color{red}{\text{SE ME HA OLVIDADO EL PARÁMETRO KERNEL REG, PQ CAMBIA??}}$

In [49]:
tf.keras.utils.set_random_seed(0)
regularizers = ["l1", "l2", "l1_l2", None]

for i in range(6):
    modelFC_optuna = models.Sequential()
    modelFC_optuna.add(layers.Dense(150, activation="relu", input_shape=(410,)))
    modelFC_optuna.add(layers.Dropout(0.1))
    modelFC_optuna.add(layers.Dense(150, activation="relu"))
    modelFC_optuna.add(layers.Dropout(0.1))
    modelFC_optuna.add(layers.Dense(150, activation="relu"))
    modelFC_optuna.add(layers.Dropout(0.1))
    modelFC_optuna.add(layers.Dense(150, activation="relu"))
    modelFC_optuna.add(layers.Dropout(0.1))
    modelFC_optuna.add(layers.Dense(2, activation="softmax"))

    modelFC_optuna.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])
    es = callbacks.EarlyStopping(monitor="val_acc", min_delta=0.01, patience=5)
    modelFC_optuna.fit(X_train, y_train_softmax, epochs=100, validation_split=0.25, callbacks=[es], verbose=0)

    # Precisión en partición de test
    loss, accuracy = modelFC_optuna.evaluate(X_test, y_test_softmax)
    print("Accuracy : {:0.2f}% ----- Regularización: {}".format(accuracy * 100, i))

1/1 [==============================] - 0s 3ms/step - loss: 0.5961 - acc: 0.8333
Accuracy : 83.33% ----- Regularización: 0
1/1 [==============================] - 0s 16ms/step - loss: 0.7155 - acc: 0.7778
Accuracy : 77.78% ----- Regularización: 1
1/1 [==============================] - 0s 31ms/step - loss: 0.6080 - acc: 0.7778
Accuracy : 77.78% ----- Regularización: 2
1/1 [==============================] - 0s 31ms/step - loss: 0.7362 - acc: 0.7222
Accuracy : 72.22% ----- Regularización: 3
1/1 [==============================] - 0s 16ms/step - loss: 0.6578 - acc: 0.7222
Accuracy : 72.22% ----- Regularización: 4
1/1 [==============================] - 0s 16ms/step - loss: 0.5682 - acc: 0.7778
Accuracy : 77.78% ----- Regularización: 5


In [55]:
tf.keras.utils.set_random_seed(0)
regularizers = ["l1", "l2", "l1_l2", None]

for regularizer in regularizers:
    modelFC_optuna = models.Sequential()
    modelFC_optuna.add(layers.Dense(150, activation="relu", input_shape=(410,)))
    modelFC_optuna.add(layers.Dropout(0.1))
    modelFC_optuna.add(layers.Dense(150, activation="relu", kernel_regularizer=regularizer))
    modelFC_optuna.add(layers.Dropout(0.1))
    modelFC_optuna.add(layers.Dense(150, activation="relu", kernel_regularizer=regularizer))
    modelFC_optuna.add(layers.Dropout(0.1))
    modelFC_optuna.add(layers.Dense(150, activation="relu", kernel_regularizer=regularizer))
    modelFC_optuna.add(layers.Dropout(0.1))
    modelFC_optuna.add(layers.Dense(2, activation="softmax"))

    modelFC_optuna.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])
    es = callbacks.EarlyStopping(monitor="val_acc", min_delta=0.01, patience=5)
    modelFC_optuna.fit(X_train, y_train_softmax, epochs=100, validation_split=0.25, callbacks=[es])

    # Precisión en partición de test
    loss, accuracy = modelFC_optuna.evaluate(X_test, y_test_softmax)
    print("Accuracy : {:0.2f}% ----- Regularización: {} \n".format(accuracy * 100, regularizer))

Epoch 1/100
2/2 [==============================] - 1s 192ms/step - loss: 47.6402 - acc: 0.5882 - val_loss: 44.8239 - val_acc: 0.5882
Epoch 2/100
2/2 [==============================] - 0s 31ms/step - loss: 44.3597 - acc: 0.5882 - val_loss: 42.5165 - val_acc: 0.5882
Epoch 3/100
2/2 [==============================] - 0s 28ms/step - loss: 42.1256 - acc: 0.6471 - val_loss: 40.6336 - val_acc: 0.5882
Epoch 4/100
2/2 [==============================] - 0s 24ms/step - loss: 40.2910 - acc: 0.6667 - val_loss: 38.9884 - val_acc: 0.5882
Epoch 5/100
2/2 [==============================] - 0s 32ms/step - loss: 38.6668 - acc: 0.7059 - val_loss: 37.5014 - val_acc: 0.5882
Epoch 6/100
1/1 [==============================] - 0s 14ms/step - loss: 36.1515 - acc: 0.5000
Accuracy : 50.00% ----- Regularización: l1 

Epoch 1/100
2/2 [==============================] - 1s 177ms/step - loss: 5.1030 - acc: 0.4118 - val_loss: 4.7266 - val_acc: 0.5882
Epoch 2/100
2/2 [==============================] - 0s 41ms/step - los

$\color{red}{\text{CÓMO PUEDE SER QUE PIERDA PRECISIÓN CON SEMILLA + KERNEL = NONE??}}$

Otra posibilidad es tratar de tener en cuenta los datos con peores resultados en el backpropagation, es decir, asignar un mayor peso a esos datos. Esta es la idea del método "AdaBoost" que sin embargo no está actualmente implementado en keras. La alternativa será definir propiamente otra función de loss dentro de por ejemplo el optimizador Adam (primera configuración de red neuronal optimizada con ``optuna`` anteriormente).

Fuente: https://stackoverflow.com/questions/48720197/weight-samples-if-incorrect-guessed-in-binary-cross-entropy

In [58]:
def custom_loss(y_true, y_pred, tp_weight=0.2, tn_weight=0.2, fp_weight=1.2, fn_weight=1.2):
    '''
    Función de pérdida personalizada para el optimizador de una red neuronal.
    El método recibe las predicciones y el valor real de la clasificación, así como los pesos que se desea asignar a los
    clasificaciones tanto erróneas como acertadas.
    '''
    # Get predictions
    y_pred_classes = tf.keras.backend.greater_equal(y_pred, 0.5)
    y_pred_classes_float = tf.keras.backend.cast(y_pred_classes, tf.keras.backend.floatx())
    y_true_float = tf.keras.backend.cast(y_true, tf.keras.backend.floatx())

    # Get misclassified examples
    wrongly_classified = tf.keras.backend.not_equal(y_true_float, y_pred_classes_float)
    wrongly_classified_float = tf.keras.backend.cast(wrongly_classified, tf.keras.backend.floatx())

    # Get correctly classified examples
    correctly_classified = tf.keras.backend.equal(y_true_float, y_pred_classes_float)
    correctly_classified_float = tf.keras.backend.cast(wrongly_classified, tf.keras.backend.floatx())

    # Get tp, fp, tn, fn
    tp = correctly_classified_float * y_true_float
    tn = correctly_classified_float * (1 - y_true_float)
    fp = wrongly_classified_float * y_true_float
    fn = wrongly_classified_float * (1 - y_true_float)

    # Get weights
    weight_tensor = tp_weight * tp + fp_weight * fp + tn_weight * tn + fn_weight * fn

    loss = tf.keras.metrics.binary_crossentropy(y_true, y_pred)
    weighted_loss = loss * weight_tensor
    return weighted_loss

In [59]:
tf.keras.utils.set_random_seed(0)

# Definir y entrenar el modelo
modelFC_CL = models.Sequential()
modelFC_CL.add(layers.Dense(200, activation="relu", input_shape=(410,)))
modelFC_CL.add(layers.Dropout(0.3))
modelFC_CL.add(layers.Dense(200, activation="relu"))
modelFC_CL.add(layers.Dropout(0.3))
modelFC_CL.add(layers.Dense(200, activation="relu"))
modelFC_CL.add(layers.Dropout(0.3))
modelFC_CL.add(layers.Dense(200, activation="relu"))
modelFC_CL.add(layers.Dropout(0.3))
modelFC_CL.add(layers.Dense(1, activation="sigmoid"))

modelFC_CL.compile(loss=custom_loss, optimizer="adam", metrics=["acc"])
es = callbacks.EarlyStopping(monitor='val_acc', min_delta=0.01, patience=5)
modelFC_CL.fit(X_train, y_train, epochs=100, validation_split=0.25, callbacks=[es])
# modelFC_CL.fit(X_train, y_train, epochs=100, validation_split=0.25, verbose=0)

# Precisión en partición de test
loss, accuracy = modelFC_CL.evaluate(X_test, y_test)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Epoch 1/100
2/2 [==============================] - 1s 202ms/step - loss: 0.4190 - acc: 0.5686 - val_loss: 0.5201 - val_acc: 0.4706
Epoch 2/100
2/2 [==============================] - 0s 22ms/step - loss: 0.4364 - acc: 0.5490 - val_loss: 0.5733 - val_acc: 0.4118
Epoch 3/100
2/2 [==============================] - 0s 30ms/step - loss: 0.2555 - acc: 0.7059 - val_loss: 0.6253 - val_acc: 0.3529
Epoch 4/100
2/2 [==============================] - 0s 29ms/step - loss: 0.3604 - acc: 0.6078 - val_loss: 0.5639 - val_acc: 0.4118
Epoch 5/100
2/2 [==============================] - 0s 33ms/step - loss: 0.3278 - acc: 0.6275 - val_loss: 0.3897 - val_acc: 0.5882
Epoch 6/100
2/2 [==============================] - 0s 30ms/step - loss: 0.1135 - acc: 0.8431 - val_loss: 0.3289 - val_acc: 0.6471
Epoch 7/100
2/2 [==============================] - 0s 23ms/step - loss: 0.2608 - acc: 0.6863 - val_loss: 0.3236 - val_acc: 0.6471
Epoch 8/100
2/2 [==============================] - 0s 31ms/step - loss: 0.1220 - acc: 0.8

Finalmente, hay una manera de definir un modelo en keras y adaptarlo para su uso en ``sklearn``, donde sí existe una función para aplicar AdaBoost.

In [64]:
def keras_to_sklearn_model():
    modelFC_AdaBoost = models.Sequential()
    modelFC_AdaBoost.add(layers.Dense(200, activation="relu", input_shape=(410,)))
    modelFC_AdaBoost.add(layers.Dropout(0.3))
    modelFC_AdaBoost.add(layers.Dense(200, activation="relu"))
    modelFC_AdaBoost.add(layers.Dropout(0.3))
    modelFC_AdaBoost.add(layers.Dense(200, activation="relu"))
    modelFC_AdaBoost.add(layers.Dropout(0.3))
    modelFC_AdaBoost.add(layers.Dense(200, activation="relu"))
    modelFC_AdaBoost.add(layers.Dropout(0.3))
    modelFC_AdaBoost.add(layers.Dense(1, activation="sigmoid"))

    modelFC_AdaBoost.compile(loss=custom_loss, optimizer="adam", metrics=["acc"])
    return modelFC_AdaBoost

In [67]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.ensemble import AdaBoostClassifier

modelFC_AdaBoost = KerasClassifier(build_fn=keras_to_sklearn_model, verbose=0)
AdaBoost_classif = AdaBoostClassifier(base_estimator=modelFC_AdaBoost)

C:\Users\saral\AppData\Local\Temp\ipykernel_15372\1299115203.py:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  modelFC_AdaBoost = KerasClassifier(build_fn=keras_to_sklearn_model, verbose=0)


In [68]:
from sklearn.metrics import accuracy_score

# Entrenar el modelo
AdaBoost_classif.fit(X_train, y_train)

# Predicción en partición de test
y_pred = AdaBoost_classif.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

ValueError: KerasClassifier doesn't support sample_weight.

Thats because the AdaboostClassifier only support those base_estimators which have a sample_weight in their fit() method.

Now although KerasClassifier supports sample_weight (from Sequential.fit()) by using the **kwargs in KerasClassifier.fit(), but scikit-learn uses a signature detecting function which checks the parameters available only in the KerasClassifier.fit()

This is what it returns:

from sklearn.utils.fixes import signature
print(signature(model.fit))
**Output: (x, y, kwargs)

As you can see that the sample_weight is not present here, so AdaBoostClassifier will throw an error. One possible workaround is to extend the KerasClassifier class and then update the fit() method to add sample_weight there.

I am not sure what can be done in the source code of KerasClassifier or in AdaboostClassifier for that case to handle this.

# Create submissions

In [9]:
import pathlib
from datetime import datetime

def create_submission(pred, method, test_id=testFNC["Id"]):
    submissionDF = pd.DataFrame(list(zip(test_id, pred)), columns=["Id", "Probability"])
    print(submissionDF.shape) # Comprobación del tamaño, debe ser: (119748, 2)
    current_time = datetime.now().strftime("%d-%m-%Y_%Hh%Mmin")
    current_path = pathlib.Path().resolve()
    parent_path = current_path.parent
    submissionDF.to_csv(f"{parent_path}\submissions\MLSP_submission_{method}_{current_time}.csv", header=True, index=False)